In [1]:
import random

import pandas as pd
import numpy as np
import math
#读入数据集
u1_base = pd.read_csv('ml-100k/u1.base', sep='\t', names=['uid', 'iid', 'rate', 'timestamp'])
u1_test = pd.read_csv('ml-100k/u1.test', sep='\t', names=['uid', 'iid', 'rate', 'timestamp'])


#定义误差函数
def ERR(rating_matrix, test):
    cnt = 0
    abs_err = 0
    squ_err = 0

    for index, row in test.iterrows():
        user_id = row['uid']
        item_id = row['iid']
        true_rating = row['rate']

        predicted_rating = rating_matrix[user_id - 1][item_id - 1]

        # 计算绝对误差/平方误差
        absolute_error = abs(predicted_rating - true_rating)
        abs_err += absolute_error

        square_error = pow(predicted_rating - true_rating, 2)
        squ_err += square_error

        cnt += 1
    # 计算平均绝对误差/平方误差
    mae = abs_err / cnt
    rmse = (squ_err / cnt) ** 0.5
    return mae, rmse


#初始化
user_num = u1_base['uid'].max()
item_num = u1_base['iid'].max()

rating_matrix = np.zeros((user_num, item_num), float)
y_ui = np.zeros((user_num, item_num), int)

#base记录转化为matrix
for index, row in u1_base.iterrows():
    user_id = row['uid']
    item_id = row['iid']
    rating = row['rate']
    rating_matrix[user_id - 1, item_id - 1] = rating
    y_ui[user_id - 1, item_id - 1] = 1

#全局平均
GlobalAverage = rating_matrix.sum() / y_ui.sum()

#计算四个参数 user_means item_means user_bias item_bias
rating_sum_row = [sum(row) for row in rating_matrix]
y_sum_row = [sum(row) for row in y_ui]

rating_sum_col = [sum(column) for column in zip(*rating_matrix)]
y_sum_col = [sum(column) for column in zip(*y_ui)]

user_means = []
for i in range(user_num):
    if y_sum_row[i] == 0:
        user_means.append(GlobalAverage)
    else:
        user_means.append(rating_sum_row[i] / y_sum_row[i])

item_means = []
for i in range(item_num):
    if y_sum_col[i] == 0:
        item_means.append(GlobalAverage)
    else:
        item_means.append(rating_sum_col[i] / y_sum_col[i])

user_bias = []
for i in range(user_num):
    if y_sum_row[i] == 0:
        user_bias.append(0)
    else:
        sum_bias = 0
        for j in range(item_num):
            sum_bias += y_ui[i][j] * (rating_matrix[i][j] - GlobalAverage)
        user_bias.append(sum_bias / y_sum_row[i])

item_bias = []
for i in range(item_num):
    if y_sum_col[i] == 0:
        item_bias.append(0)
    else:
        sum_bias = 0
        for j in range(user_num):
            sum_bias += y_ui[j][i] * (rating_matrix[j][i] - GlobalAverage)
        item_bias.append(sum_bias / y_sum_col[i])

In [3]:
#PSVD
rating_matrix_psvd = np.copy(rating_matrix)
#预处理
for i in range(user_num):
    for k in range(item_num):
        if rating_matrix_psvd[i, k] != 0:
            rating_matrix_psvd[i, k] -= user_means[i]
# 计算SVD
U, sigma, Vt = np.linalg.svd(rating_matrix_psvd, full_matrices=False)
sigma = np.diag(sigma)
k = 20  # 选择前k个奇异值
U_k = U[:, :k]
sigma_k = sigma[:k, :k]
Vt_k = Vt[:k, :]
# 重构评分矩阵
R_approx = np.dot(U_k, np.dot(sigma_k, Vt_k))
for idx, row in u1_test.iterrows():
    user_id = row['uid'] - 1
    item_id = row['iid'] - 1
    R_approx[user_id, item_id] += user_means[user_id]
    if R_approx[user_id, item_id] > 5:
        R_approx[user_id, item_id] = 5
    if R_approx[user_id, item_id] < 1:
        R_approx[user_id, item_id] = 1
mae1, rmse1 = ERR(R_approx, u1_test)
print(f'(RMSE:{rmse1:.4f},MAE:{mae1:.4f})')

(RMSE:1.0173,MAE:0.8061)


In [4]:
#超参数
alpha_u = alpha_v = beta_u = beta_v = 0.1
lr = 0.01
d = 20

In [4]:
#ALS

# U = np.random.rand(user_num, d)
# V = np.random.rand(item_num, d)
# U = (U - 0.5) * 0.01
# V = (V - 0.5) * 0.01
# for t in range(epoch):
#     print(f'\r第{t+1}轮', end='')
#     for u in range(user_num):
#         b_u = np.zeros((1, d), float)
#         A_u = np.zeros((d, d), float)
#         for i in range(item_num):
#             b_u += y_ui[u, i] * rating_matrix[u, i] * V[i]
#             A_u += y_ui[u, i] * (V[i].T @ V[i] + alpha_u * np.eye(d))
#         #更新
#         if np.linalg.det(A_u) != 0:
#             U[u] = b_u @ np.linalg.inv(A_u)
#     for k in range(item_num):
#         b_i = np.zeros((1, d), float)
#         A_i = np.zeros((d, d), float)
#         for i in range(user_num):
#             b_i += y_ui[i, k] * rating_matrix[i, k] * U[i]
#             A_i += y_ui[i, k] * (U[i].T @ U[i] + alpha_v * np.eye(d))
#         #更新
#         if np.linalg.det(A_i) != 0:
#             V[k] = b_i @ np.linalg.inv(A_i)

In [5]:
#ALS

# rating_matrix_als = np.copy(rating_matrix)
# for i in range(user_num):
#     for k in range(item_num):
#         if rating_matrix_als[i, k] == 0:
#             r_hat = np.dot(U[i], V[k])
#             rating_matrix_als[i, k] = r_hat
# mae2, rmse2 = ERR(rating_matrix_als, u1_test)
# print(f'(RMSE:{rmse2:.4f},MAE:{mae2:.4f})')

In [5]:
#SGD
U = np.copy(U_k)
V = np.copy(Vt_k).T
epoch = 100
for t in range(epoch):
    for n in range(len(u1_base)):
        print(f'\r{t+1}:{n+1}', end='')
        random_num = random.randint(1, len(u1_base))
        row = u1_base.iloc[random_num - 1]
        user_id = row['uid'] - 1
        item_id = row['iid'] - 1
        rating = row['rate']
        #计算梯度
        delta_Uu = -(rating - U[user_id] @ V[item_id].T) * V[item_id] + alpha_u * U[user_id]
        delta_Vi = -(rating - U[user_id] @ V[item_id].T) * U[user_id] + alpha_v * V[item_id]
        #update
        U[user_id] = U[user_id] - lr * delta_Uu
        V[item_id] = V[item_id] - lr * delta_Vi
    lr *= 0.9

100:80000

In [6]:
#SGD
rating_matrix_sgd = np.copy(rating_matrix)
for idx, row in u1_test.iterrows():
    user_id = row['uid'] - 1
    item_id = row['iid'] - 1
    r_hat = np.dot(U[user_id], V[item_id])
    if r_hat > 5:
        r_hat = 5
    if r_hat < 1:
        r_hat = 1
    rating_matrix_sgd[user_id,item_id] = r_hat
mae3, rmse3 = ERR(rating_matrix_sgd, u1_test)
print(f'(RMSE:{rmse3:.4f},MAE:{mae3:.4f})')

(RMSE:0.9470,MAE:0.7481)


In [7]:
#RSVD
def R_HAT_RSVD(u, i, U, V, bu, bi, miu):
    return U[u] @ V[i].T + bu[u] + bi[i] + miu

In [8]:
#RSVD
U = np.random.rand(user_num + 1, d)
V = np.random.rand(item_num + 1, d)
U = (U - 0.5) * 0.01
V = (V - 0.5) * 0.01
miu = GlobalAverage
for t in range(epoch):
    loss = 0
    for n in range(len(u1_base)):
        random_num = random.randint(1, len(u1_base))
        row = u1_base.iloc[random_num - 1]
        user_id = row['uid'] - 1
        item_id = row['iid'] - 1
        rating = row['rate']

        R_hat = R_HAT_RSVD(user_id, item_id, U, V, user_bias, item_bias, miu)
        #损失函数
        loss += 0.5 * (rating - R_hat) ** 2 + 0.5 * alpha_u * (U[user_id] @ U[user_id].T) + 0.5 * alpha_v * (
                V[item_id] @ V[item_id].T) + 0.5 * beta_u * (user_bias[user_id]) ** 2 + 0.5 * beta_v * (
                    item_bias[item_id]) ** 2

        e_ui = rating - R_hat
        #计算梯度
        delta_miu = -e_ui
        delta_bu = -e_ui + beta_u * user_bias[user_id]
        delta_bi = -e_ui + beta_v * item_bias[item_id]
        delta_Uu = -e_ui * V[item_id] + alpha_u * U[user_id]
        delta_Vi = -e_ui * U[user_id] + alpha_v * V[item_id]

        #update
        miu = miu - lr * delta_miu
        user_bias[user_id] = user_bias[user_id] - lr * delta_bu
        item_bias[item_id] = item_bias[item_id] - lr * delta_bi
        U[user_id] = U[user_id] - lr * delta_Uu
        V[item_id] = V[item_id] - lr * delta_Vi
    lr *= 0.9
    print(f'\r第{t + 1}轮loss:{loss}', end='')

第100轮loss:36656.91483998905

In [9]:
#RSVD
rating_matrix_rsvd = np.copy(rating_matrix)
for idx, row in u1_test.iterrows():
    user_id = row['uid'] - 1
    item_id = row['iid'] - 1
    r_hat = miu + user_bias[user_id] + item_bias[item_id] + U[user_id] @ V[item_id].T 
    if r_hat > 5:
        r_hat = 5
    if r_hat < 1:
        r_hat = 1
    rating_matrix_rsvd[user_id,item_id] = r_hat
mae4, rmse4 = ERR(rating_matrix_rsvd, u1_test)
print(f'(RMSE:{rmse4:.4f},MAE:{mae4:.4f})')

(RMSE:0.9772,MAE:0.7663)
